In [6]:
import glob
import os

import geopandas
import pandas

# typedefs
T_DataFrame = pandas.core.frame.DataFrame
T_GeoDataFrame = geopandas.geodataframe.GeoDataFrame

# aggregate data for the commute graph

In [7]:
def concat_all(geometry: str) -> T_DataFrame:
    return pandas.concat([
        pandas.read_csv(path)\
            .assign(quantity=os.path.basename(path).replace(".csv", ""))
        for path in glob.glob("output/webapp/{}/*".format(geometry))
    ])

df_segments = concat_all("segments")
df_stations = concat_all("stations")

In [8]:
df_segments.to_csv("output/webapp/segments.csv", index=False)
df_stations.to_csv("output/webapp/stations.csv", index=False)

In [9]:
gdf_segments = geopandas.read_file("analysis/geometries/segments.geojson").to_crs({"init": "epsg:4326"})
gdf_stations = geopandas.read_file("analysis/geometries/stations.geojson").to_crs({"init": "epsg:4326"})

# aggregate data in geojson format for the system graph

In [10]:
ALL_INDEX_COLUMNS = {"STATION", "START_STATION", "END_STATION", "ROUTE", "LINE"}

def flatten_and_merge(df: T_DataFrame, gdf: T_GeoDataFrame) -> T_GeoDataFrame:
    index_columns = list(ALL_INDEX_COLUMNS.intersection(df.columns))
    gdf_flattened = geopandas.GeoDataFrame(
        pandas.merge(
            gdf,
            df.pivot_table(
                index=index_columns,
                values="center",
                columns="quantity"
            ).reset_index(),
            on=index_columns
        )
    )
    gdf_flattened["length_mi"] = gdf.to_crs({"init": "epsg:2249"}).length / 5280 # NAD83 MA ftUS
    gdf_flattened.crs = gdf.crs
    return gdf_flattened

flatten_and_merge(df_segments, gdf_segments).to_file("output/webapp/segments-centers.geojson", driver="GeoJSON")
flatten_and_merge(df_stations, gdf_stations).to_file("output/webapp/stations-centers.geojson", driver="GeoJSON")

# calculation of reference values for particulates

In [ ]:
here, we are using the formula provided by 